# Google Trends data cleaning

In [203]:
import pandas as pd
import numpy as np
import csv

In [204]:
df_yoo=pd.DataFrame({(2,2),(11,2)},{1,2})
df_yoo

,0,1
1,11,2
2,2,2


In [205]:
# Getting the code
df_google = pd.DataFrame()
file_names = ['Gtrend_data_untidy/Gtrend_protest_data.csv', 'Gtrend_data_untidy/Gtrend_revolution_data.csv', 'Gtrend_data_untidy/Gtrend_riots_data.csv', 'Gtrend_data_untidy/Gtrend_strike_data.csv', 'Gtrend_data_untidy/Gtrend_violence_data.csv', 'Gtrend_data_untidy/Gtrend_unrest_data.csv']
for file_name in file_names:
    with open(file_name, 'r') as file:
        next(file)
        data = csv.reader(file)
        df0 = pd.DataFrame(data)
        df_google = pd.concat([df_google, df0])

# Reading matching data
with open("country_codes.csv", 'r') as file:
    next(file)
    data = csv.reader(file)
    countries_df = pd.DataFrame(data)
countries_df.columns = column_names = ["country", "geo", "country_code", "number"]
countries_df = countries_df.drop(columns="number")

df_google

,0,1,2,3,4,5,6,7
0,1,2004-01-01,0,protest,AF,all,web,0
1,2,2004-02-01,0,protest,AF,all,web,0
2,3,2004-03-01,100,protest,AF,all,web,0
3,4,2004-04-01,0,protest,AF,all,web,0
4,5,2004-05-01,0,protest,AF,all,web,0
...,...,...,...,...,...,...,...,...
28285,28286,2022-10-01,0,unrest,ZW,all,web,0
28286,28287,2022-11-01,<1,unrest,ZW,all,web,0
28287,28288,2022-12-01,0,unrest,ZW,all,web,0
28288,28289,2023-01-01,0,unrest,ZW,all,web,0


In [206]:
# labelling and dropping columns
df_google.columns = column_names = ["code", "date", "hits", "keyword", "geo", "time", "gprop", "category"]
df_google = df_google.drop(columns=["code","time", "gprop", "category"])
df_google["code"] = df_google["date"] + "/" + df_google["geo"]
df_google = df_google.replace(to_replace="<1", value=0.5) # replace "<1" values with "0.5"

# day month year
df_google["date"] = pd.to_datetime(df_google["date"])
df_google["day"] = df_google["date"].dt.day
df_google["month"] = df_google["date"].dt.month
df_google["year"] = df_google["date"].dt.year
df_google = df_google.drop(columns=["day"])

# matching on country codes
merged_df_google = pd.DataFrame.merge(df_google, countries_df)
merged_df_google

# selecting only the years we are interested in; i.e. pre-2019
merged_df_google = merged_df_google[merged_df_google["year"] <= 2019]

# pivot
pivot_df = merged_df_google.pivot(index = "code", columns = "keyword", values = "hits")
pivot_df = pivot_df.reset_index() # reset index
pivot_df[["date","geo"]] = pivot_df["code"].str.split("/", expand=True)

# matching on country codes
pivot_df = pd.DataFrame.merge(pivot_df, countries_df, left_on= "geo", right_on="geo")
pivot_df

pivot_df.to_csv(r'google_all_data_monthly.csv')
pivot_df

,code,protest,revolution,riots,strike,unrest,violence,date,geo,country,country_code
0,2004-01-01/AD,0,0,NaN,0,NaN,NaN,2004-01-01,AD,Andorra,AND
1,2004-02-01/AD,0,0,NaN,100,NaN,NaN,2004-02-01,AD,Andorra,AND
2,2004-03-01/AD,0,0,NaN,0,NaN,NaN,2004-03-01,AD,Andorra,AND
3,2004-04-01/AD,100,100,NaN,0,NaN,NaN,2004-04-01,AD,Andorra,AND
4,2004-05-01/AD,0,0,NaN,67,NaN,NaN,2004-05-01,AD,Andorra,AND
...,...,...,...,...,...,...,...,...,...,...,...
42811,2019-08-01/ZW,2,4,1,3,1,5,2019-08-01,ZW,Zimbabwe,ZWE
42812,2019-09-01/ZW,1,4,1,9,0,10,2019-09-01,ZW,Zimbabwe,ZWE
42813,2019-10-01/ZW,0,4,0,7,0.5,9,2019-10-01,ZW,Zimbabwe,ZWE
42814,2019-11-01/ZW,0,3,2,8,0,11,2019-11-01,ZW,Zimbabwe,ZWE


In [207]:
# preparing to taking yearly average
pivot_df["date"] = pd.to_datetime(pivot_df["date"])
variables = ["protest","revolution","riots","strike","unrest","violence"]
for variable in variables:
    pivot_df[variable] = pd.to_numeric(pivot_df[variable], errors='coerce') # non-numerical values converted to NaN
pivot_df.set_index("date", inplace=True) # set date column as index
pivot_df = pivot_df.groupby("country_code")

# taking yearly average
df_google_yearly = pd.DataFrame()
for variable in variables:
    df_google_yearly[variable] = pivot_df[variable].resample("Y",loffset="-6m").mean() #resampling — to date at the middle of the year

df_google_yearly = df_google_yearly.reset_index() # reset index
df_google_yearly["date"] = pd.to_datetime(df_google_yearly["date"]) + pd.Timedelta(days=1) # shift by one day
df_google_yearly.to_csv(r'google_all_data_yearly.csv')
df_google_yearly

/var/folders/vv/q11k4tkx2bbdz6n3fss5p7j00000gn/T/ipykernel_8485/1873302205.py:12: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  df_google_yearly[variable] = pivot_df[variable].resample("Y",loffset="-6m").mean() #resampling
/var/folders/vv/q11k4tkx2bbdz6n3fss5p7j00000gn/T/ipykernel_8485/1873302205.py:12: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  df_google_yearly[variable] = pivot_df[variable].resample("Y",loffset="-6m").mean() #resampling
/var/folders/vv/q11k4tkx2bbdz6n3fss5p7j00000gn/T/ipykernel_8485/1873302205.py:12: FutureWarn

,country_code,date,protest,revolution,riots,strike,unrest,violence
0,ABW,2004-07-01,22.000000,0.000000,0.000000,13.166667,NaN,15.416667
1,ABW,2005-07-01,13.000000,4.250000,12.750000,7.000000,NaN,7.166667
2,ABW,2006-07-01,4.250000,11.416667,6.333333,5.250000,NaN,2.666667
3,ABW,2007-07-01,0.416667,15.833333,0.000000,1.666667,NaN,2.416667
4,ABW,2008-07-01,1.083333,1.916667,3.000000,0.750000,NaN,0.583333
...,...,...,...,...,...,...,...,...
3563,ZWE,2015-07-01,0.500000,8.250000,0.416667,3.416667,0.208333,6.166667
3564,ZWE,2016-07-01,1.583333,8.500000,2.916667,4.250000,0.333333,8.666667
3565,ZWE,2017-07-01,0.791667,6.583333,0.750000,3.166667,0.458333,6.916667
3566,ZWE,2018-07-01,0.625000,5.166667,0.333333,5.583333,0.458333,8.250000
